In [1]:
import numpy as np
import onnxruntime as ort
import time
from onnxruntime.quantization import QuantFormat, QuantType, quantize_static
import cv2
import os
import matplotlib.pyplot as plt
from onnxruntime.quantization import CalibrationDataReader
from tqdm import tqdm

In [2]:
def _preprocess_images(images_folder: str):
    providers = ['DmlExecutionProvider']
    options = ort.SessionOptions()
    options.enable_mem_pattern = False
    options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
    options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
    decomposer_sess = ort.InferenceSession('./preprocessed/decomposer.onnx', sess_options=options, providers=providers)
    combiner_sess = ort.InferenceSession('./preprocessed/combiner.onnx', sess_options=options, providers=providers)
    morpher_sess = ort.InferenceSession('./preprocessed/morpher.onnx', sess_options=options, providers=providers)
    rotator_sess = ort.InferenceSession('./preprocessed/rotator.onnx', sess_options=options, providers=providers)
    batch_filenames = os.listdir(images_folder)
    batch_data = []
    for image_name in tqdm(batch_filenames):
        image_filepath = os.path.join(images_folder, image_name)
        img = cv2.imread(image_filepath, cv2.IMREAD_UNCHANGED)
        if img is None:
            continue
        img = cv2.resize(img, (450,900),interpolation = cv2.INTER_LANCZOS4)
        padding_img = np.zeros((1024,1024,4),np.uint8)
        padding_img[40:40+900,287:287+450,:] = img
        img = cv2.resize(padding_img, (512,512),interpolation = cv2.INTER_LANCZOS4)

        decomposer_res = decomposer_sess.run(None, {'input_image':img})
        for i in range(2): #Repeat 2 times
            eyebrow_pose = np.zeros((1,12), np.float32)
            rand_idx = int(np.random.randint(low = 0, high = 5))
            eyebrow_pose[0, 2 * rand_idx] = np.random.default_rng().uniform(0.0,1.0)
            eyebrow_pose[0, 2 * rand_idx + 1] = np.random.default_rng().uniform(0.0,1.0)
            combiner_inp = {'image_prepared':decomposer_res[2], 
                            'eyebrow_background_layer':decomposer_res[0], 
                            'eyebrow_layer':decomposer_res[1], 
                            'eyebrow_pose':eyebrow_pose}
            combiner_res = combiner_sess.run(None, combiner_inp)
            for j in range(2):
                face_pose = np.zeros((1,27),np.float32)
                rand_idx = int(np.random.randint(low = 0, high = 5))
                face_pose[0, 2 * rand_idx] = np.random.default_rng().uniform(0.0,1.0)
                face_pose[0, 2 * rand_idx + 1] = np.random.default_rng().uniform(0.0,1.0)
                rand_idx = int(np.random.randint(low = 12, high = 22))
                face_pose[0, rand_idx] = np.random.default_rng().uniform(0.0,1.0)
                face_pose[0, 23] = np.random.default_rng().uniform(0.0,1.0)
                face_pose[0, 24] = np.random.default_rng().uniform(0.0,1.0)
                face_pose[0, 25] = np.random.default_rng().uniform(-1.0,1.0)
                face_pose[0, 26] = np.random.default_rng().uniform(-1.0,1.0)
                morpher_inp = {
                    'image_prepared':decomposer_res[2],
                    'im_morpher_crop':combiner_res[0],
                    'face_pose':face_pose,
                    '/face_morpher/downsample_blocks.3/downsample_blocks.3.2/Relu_output_0':combiner_res[1]
                }
                morpher_res = morpher_sess.run(None, morpher_inp)
                for k in range(2):
                    body_pose = np.random.uniform(-1.0,1.0,[1,6]).astype(np.float32)
                    body_pose[0,5] = np.random.default_rng().uniform(0.0,1.0)
                    rotator_inp = {
                        'face_morphed_half':morpher_res[1],
                        'rotation_pose':body_pose
                    }
                    rotator_res = rotator_sess.run(None, rotator_inp)
                    editor_inp = {
                        'morphed_image' : morpher_res[0],
                        'rotated_warped_image' : rotator_res[0],
                        'rotated_grid_change' : rotator_res[1],
                        'rotation_pose':body_pose
                    }
                    batch_data.append(editor_inp)
    return batch_data

In [3]:
class EditorDataReader(CalibrationDataReader):
    def __init__(self, calibration_image_folder: str):
        self.enum_data = None

        # Convert image to input data
        self.data_list = _preprocess_images(
            calibration_image_folder
        )
        self.datasize = len(self.data_list)

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(self.data_list)
        return next(self.enum_data, None)

    def rewind(self):
        self.enum_data = None
dr = EditorDataReader('Z:/ComfyUI-aki-v1.5/output/')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 298/298 [00:32<00:00,  9.21it/s]


In [4]:
quantize_static(
        './preprocessed/editor.onnx',
        './quantized/editor.onnx',
        dr,
        quant_format=QuantFormat.QDQ,
        per_channel=True,
        weight_type=QuantType.QInt8,
        nodes_to_exclude = ['/Mul','/Mul_1','/Add','/Mul_6'],
        extra_options = {
            'ActivationSymmetric':True,
            'QuantizeBias':False
        }
    )

['Gemm', 'InstanceNormalization', 'Slice', 'AveragePool', 'Softmax', 'GatherElements', 'BatchNormalization', 'MaxPool', 'Where', 'Transpose', 'Gather', 'Reshape', 'Mul', 'Pad', 'Unsqueeze', 'Resize', 'Add', 'GlobalAveragePool', 'Sigmoid', 'ArgMax', 'EmbedLayerNormalization', 'LeakyRelu', 'Conv', 'Squeeze', 'ConvTranspose', 'Split', 'Concat', 'Relu', 'LayerNormalization', 'MatMul', 'Clip']
 16
com.microsoft.nchwc 1
ai.onnx.ml 5
ai.onnx.training 1
ai.onnx.preview.training 1
com.microsoft 1
com.microsoft.experimental 1
org.pytorch.aten 1
com.microsoft.dml 1
[domain: ""
version: 16
]


In [5]:
non_quantized_session =ort.InferenceSession('./preprocessed/editor.onnx', None)
quantized_session =ort.InferenceSession('./quantized/editor.onnx', None)

In [6]:
dr.rewind()
inp = dr.get_next()
non_res =  non_quantized_session.run(None, inp)
qt_res = quantized_session.run(None, inp)
((qt_res[1] -non_res[1])**2).mean()

np.float64(90.26974487304688)